### Topic Modeling: Demand for Cybersecurity Professionals  

In [2]:
import pandas as pd 
import numpy as np
import nltk 
from nltk.corpus import stopwords 
import re
import string

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity

#### Data Preprocessing 
Indeed Data 

In [11]:
df = pd.read_csv("C:/Users/hanna/Scape-Save-DAEN690/Datasets/indeed_cleaned.csv.gz")

In [12]:
#Remove duplicate postings if the job title, description and location are the same 
df.drop_duplicates(subset=['job_title', 'description_text', 'location'], inplace=True)

In [13]:
#Set to lower case so capitalized titles don't get counted separately
df['job_title'] = df['job_title'].apply(lambda x: x.lower())

In [17]:
#obtaining count of 'cybersecurity' roles within dataset 
cyber_roles = list(df[(df['job_title'].str.contains('cyber'))  | (df['job_title'].str.contains('security'))]['job_title'].values)
len(cyber_roles)

['cloud security engineer', 'information security associate manager', 'senior cyber security account executive (remote us)', 'lead it security engineer', 'information security specialist - information technology (multiple locations)', 'information security analyst', 'information systems security engineer', 'software development engineer, aws security', 'information system security officer (isso)', 'cyber defense analyst iii', 'information security specialist', 'security automation engineer - location negotiable', 'system security administrator', 'senior cybersecurity incident response analyst', 'managing principal - cybersecurity', 'cybersecurity systems analyst', 'information security trainee', 'information security analyst', 'cloud application security architect', 'cybersecurity data security - senior - consulting - location open', 'assistant/associate professor of cyber intelligence and security', 'collateral cyber security branch head and information systems security manager (issm)

In [51]:
#creating new dataframe with 'cyber' and 'security' as keywords in job title
       
new_df = df[df["job_title"].apply(lambda x: any(k in x for k in cyber_roles))]
new_df.shape    

In [2]:
def role_type(x):
    if 'lead' in x:
        return 'senior'
    elif 'senior' in x:
        return 'senior'
    elif 'sr' in x:
        return 'senior'
    elif 'chief' in x:
        return 'upper mgmt'
    elif 'manager' in x:
        return 'upper mgmt'
    elif 'principal' in x:
        return 'upper mgmt'
    elif 'junior' in x:
        return 'entry-level'
    elif 'jr' in x:
        return 'entry-level'
    elif 'entry-level' in x:
        return 'entry-level'
    elif 'entry level' in x:
        return 'entry-level'
    elif 'professor' in x: 
        return 'academia'
    else:
        return 'regular'
    
new_df['role_type'] = new_df['job_title'].apply(role_type)

#Can be used for visualizations 
job_df = new_df.groupby('role_type').count().reset_index()[['role_type', 'job_title']]


NameError: name 'new_df' is not defined

In [61]:
corpus= new_df['description_text']

count_vect = CountVectorizer()

x_counts = count_vect.fit_transform(corpus)

cos_df = pd.DataFrame(cosine_similarity(x_counts))

#Creating two indexs to compare job descriptions and calculate similarity 
i, j = np.indices(cos_df.shape).reshape(2, -1)

cos_values = cos_df.values.reshape(-1)

#Creating a dataframe with indexes and similarity score 
cos_sim_df = pd.DataFrame({'i': i, 'j': j, 'score':cos_values})

#similarity score >0.99 indicates that the job descriptions are very similar with slight variations in wordings
cos_rem = cos_sim_df[(cos_sim_df['sim']>0.99)&(i!=j)]

#Method to remove duplicates but keep first instance. Trying to drop duplicates on i and j column wouldn't work as the numbers don't duplicate in those
#columns. Setting another column that combines their values ensure that duplicates can be dropped.

cos_rem['i*j'] = cos_rem['i'] * cos_rem['j']
drop_rows = np.unique(cos_rem.drop_duplicates(subset=['i*j'], keep='first')['i'].values)

#Drop from original dataframe
new_df = new_df[~new_df.index.isin(drop_rows)]
new_df = new_df.reset_index()
new_df.drop(columns={'index'}, inplace=True)

C:\Users\hanna\AppData\Local\Temp\ipykernel_26032\4204453953.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cos_rem['i*j'] = cos_rem['i'] * cos_rem['j']


CareerOneStop Data

In [3]:
cos_df = pd.read_excel("C:/Users/hanna/Scape-Save-DAEN690/Datasets/careeronestop_data.xlsx") 

In [9]:
cos_df.iloc[[5230]]

,keyword,job_title,company,location,date_posted,job_link,description
5230,Network Security Architect,Sr. Security Architect,iCIMS,"Carson City, NV",03/03/2023,https://www.careeronestop.org/Toolkit/Jobs/fin...,Job Overview As a Sr. Security Architect you w...


In [5]:
cos_df.head()
cos_df.shape

(11806, 7)

In [6]:
cos_df.drop_duplicates(subset=['job_title', 'description', 'location'], inplace=True)
#Set to lower case so capitalized titles don't get counted separately
cos_df['job_title'] = cos_df['job_title'].apply(lambda x: str(x).lower())

In [7]:
#apply function to add job roles 
cos_df['role_type'] = cos_df['job_title'].apply(role_type)

job_df = cos_df.groupby('role_type').count().reset_index()[['role_type', 'job_title']]

In [11]:
corpus= cos_df['description']

count_vect = CountVectorizer()

x_count_cos = count_vect.fit_transform(corpus.values.astype('U'))

cosi_df = pd.DataFrame(cosine_similarity(x_count_cos))

#Creating two indexs to compare job descriptions and calculate similarity 
i, j = np.indices(cosi_df.shape).reshape(2, -1)

cos_values = cosi_df.values.reshape(-1)

#Creating a dataframe with indexes and similarity score 
cos_sim_df = pd.DataFrame({'i': i, 'j': j, 'score':cos_values})

#similarity score >0.99 indicates that the job descriptions are very similar with slight variations in wordings 
cos_rem = cos_sim_df[(cos_sim_df['sim']>0.99)&(i!=j)]

#Method to remove duplicates but keep first instance. Trying to drop duplicates on i and j column wouldn't work as the numbers don't duplicate in those
#columns. Setting another column that combines their values ensure that duplicates can be dropped.

cos_rem['i*j'] = cos_rem['i'] * cos_rem['j']
drop_rows = np.unique(cos_rem.drop_duplicates(subset=['i*j'], keep='first')['i'].values)

#Drop from original dataframe
cos_df = cos_df[~cos_df.index.isin(drop_rows)]
cos_df = cos_df.reset_index()
cos_df.drop(columns={'index'}, inplace=True)

C:\Users\hanna\AppData\Local\Temp\ipykernel_34584\480341193.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cos_rem['i*j'] = cos_rem['i'] * cos_rem['j']


In [23]:
cos_re = cos_sim_df[(cos_sim_df['sim']>0.96)&(i!=j)]
cos_re.to_csv("C:/Users/hanna/OneDrive/Documents/DAEN 690/cos.csv", encoding='utf-8')

In [27]:
cos_df.iloc[295:316]

,keyword,job_title,company,location,date_posted,job_link,description
295,Penetration Tester,Public Notice for Information Technology Cyber...,Federal Aviation Administration,May be filled in various FAA duty locations,11/16/2022,https://www.careeronestop.org/Toolkit/Jobs/fin...,Summary Direct hiring authority for 2210 serie...
296,Penetration Tester,Cmm Inspector,Aerotek,"Albany, OR",03/01/2023,https://www.careeronestop.org/Toolkit/Jobs/fin...,Description: Is responsible for reviewing prod...
297,Penetration Tester,Application Security Tester,General Dynamics Information Technology,"Fairfax, VA",03/14/2023,https://www.careeronestop.org/Toolkit/Jobs/fin...,Req ID: RQ143219 Type of Requisition: Regula...
298,Penetration Tester,General Inquiry – Engineer/Scientist,"Nou Systems, Inc.","Huntsville, AL",05/20/2022,https://www.careeronestop.org/Toolkit/Jobs/fin...,"nou Systems, Inc. (nSI) is here to solve diver..."
299,Penetration Tester,Computer Exploitation Engineer VAD (Red Team),"Yulista Management Services, Inc","Fort Belvoir, VA",02/18/2023,https://www.careeronestop.org/Toolkit/Jobs/fin...,"locations : FT Belvoir, VA ```{=html} ``` ti..."
300,Penetration Tester,Cyber Threat Hunt Specialist,"Nou Systems, Inc.","Huntsville, AL",01/06/2022,https://www.careeronestop.org/Toolkit/Jobs/fin...,"nou Systems, I nc. (nSI) is here to solve div..."
301,Penetration Tester,Construction Engineering Inspector/Tester,WSP USA,"Dallas, TX",10/07/2022,https://www.careeronestop.org/Toolkit/Jobs/fin...,Construction Engineering Inspector/Tester Req ...
302,Penetration Tester,"Software Test Engineer, SE-3 , TGCC *Sign On B...","Oasis Systems, LLC","San Antonio, TX",09/30/2022,https://www.careeronestop.org/Toolkit/Jobs/fin...,Overview Oasis Systems has an exciting opportu...
303,Penetration Tester,Systems Vulnerability Assessment Lead,"GCubed Enterprises, Inc.","Washington, DC",11/17/2022,https://www.careeronestop.org/Toolkit/Jobs/fin...,Systems Vulnerability Assessment Lead Location...
304,Penetration Tester,Cyber Engineering Intern - Summer 2023,CACI International,"Reston, VA",09/21/2022,https://www.careeronestop.org/Toolkit/Jobs/fin...,Cyber Engineering Intern - Summer 2023 Job Cat...


#### BERTopic Algorithm

In [6]:
from umap import UMAP
from hdbscan import HDBSCAN
from bertopic import BERTopic 
from sentence_transformers import SentenceTransformer
from sklearn.feature_extraction.text import CountVectorizer
from bertopic.representation import MaximalMarginalRelevance
from bertopic.vectorizers import ClassTfidfTransformer

# Step 1 - Extract embeddings
embedding_model = SentenceTransformer("all-MiniLM-L6-v2")

# Step 2 - Reduce dimensionality
umap_model = UMAP(n_neighbors=15, n_components=5, min_dist=0.0, metric='cosine')

# Step 3 - Cluster reduced embeddings
hdbscan_model = HDBSCAN(min_cluster_size=15, metric='euclidean', cluster_selection_method='eom', prediction_data=True)

# Step 4 - Tokenize topics
vectorizer_model = CountVectorizer(stop_words="english")

# Step 5 - Create topic representation
ctfidf_model = ClassTfidfTransformer()

# Step 6 - Fine-tune topic representations with 
# reduces redundacy and improve diversity of keywords using MMR 
representation_model = MaximalMarginalRelevance(0.5)

# All steps together with indeed data 
topic_model = BERTopic(
  embedding_model=embedding_model,          # Step 1 - Extract embeddings
  umap_model=umap_model,                    # Step 2 - Reduce dimensionality
  hdbscan_model=hdbscan_model,              # Step 3 - Cluster reduced embeddings
  vectorizer_model=vectorizer_model,        # Step 4 - Tokenize topics
  ctfidf_model=ctfidf_model,                # Step 5 - Extract topic words
  representation_model=representation_model) # Step 6 - Fine-tune topic represenations


In [7]:
cos_df.shape

(8920, 8)

In [8]:
#topic_model.fit_transform(new_df['description_text'])
topic_model.fit_transform(cos_df['description'].values.astype('U'))

([-1,
  -1,
  75,
  15,
  -1,
  1,
  -1,
  51,
  51,
  51,
  51,
  51,
  51,
  51,
  51,
  51,
  51,
  51,
  -1,
  51,
  51,
  51,
  51,
  51,
  51,
  51,
  51,
  51,
  51,
  51,
  51,
  51,
  51,
  51,
  51,
  51,
  51,
  51,
  51,
  51,
  51,
  51,
  51,
  51,
  51,
  51,
  51,
  51,
  51,
  51,
  51,
  51,
  51,
  51,
  51,
  51,
  51,
  51,
  51,
  51,
  65,
  70,
  -1,
  110,
  110,
  110,
  110,
  110,
  110,
  110,
  110,
  -1,
  110,
  110,
  65,
  110,
  -1,
  65,
  136,
  -1,
  -1,
  136,
  136,
  136,
  65,
  -1,
  -1,
  13,
  13,
  13,
  13,
  13,
  13,
  13,
  13,
  13,
  13,
  13,
  13,
  13,
  10,
  12,
  -1,
  47,
  -1,
  65,
  118,
  118,
  118,
  118,
  118,
  118,
  10,
  118,
  118,
  118,
  65,
  -1,
  -1,
  -1,
  37,
  62,
  62,
  -1,
  65,
  62,
  -1,
  62,
  70,
  65,
  62,
  62,
  62,
  37,
  -1,
  65,
  65,
  85,
  85,
  120,
  -1,
  -1,
  155,
  118,
  -1,
  -1,
  -1,
  -1,
  67,
  64,
  -1,
  65,
  -1,
  65,
  -1,
  120,
  -1,
  -1,
  -1,
  -1,
  -1,
  13,
 

In [9]:
topic_model.visualize_barchart(top_n_topics=5)

In [81]:
freq = topic_model.get_topic_info() 
freq

,Topic,Count,Name
0,-1,2409,-1_experience_cybersecurity_management_support
1,0,334,0_dod_rmf_requirements_clearance
2,1,257,1_faculty_students_umgc_curriculum
3,2,159,2_management_information_cybersecurity_experience
4,3,152,3_threat_analyst_data_cybersecurity
...,...,...,...
78,77,17,77_mtsi_employee_vesting_deductible
79,78,16,78_van_het_voor_zijn
80,79,16,79_tilly_baker_audit_risks
81,80,16,80_peraton_agencies_development_cyber


In [ ]:
#keywords present in our top topics 
topic_model.get_topic(freq.iloc[3]["Topic"])